# Introducción:

En esta lección, exploraremos cómo utilizar SQL en conjunción con Python para llevar a cabo operaciones de extracción de datos. Aprenderemos a conectarnos a bases de datos, ejecutar consultas SQL para recuperar información específica, y luego procesar y utilizar esos datos dentro del entorno de Python. Ya sea que desees obtener información de una base de datos empresarial, extraer datos para análisis o realizar tareas de ETL (Extract, Transform, Load), esta lección te proporcionará las habilidades necesarias para realizar estas tareas de manera efectiva y eficiente.

#  Conexión a una Base de Datos MySQL desde Python:

En esta sección, aprenderemos cómo establecer una conexión con una base de datos MySQL existente desde un script de Python utilizando el constructor de la clase `connect()`.

El constructor `connect()` es esencial para crear una conexión con un servidor MySQL y devuelve un objeto de tipo `MySQLConnection`. A través de un ejemplo práctico, exploraremos cómo utilizarlo de manera efectiva.

🚨🚨 **NOTA IMPORTANTE** 🚨🚨 Si encuentras un error al ejecutar la celda a continuación, sigue estos pasos en tu terminal:

1. Ejecuta `pip install mysql-connector`.

2. Luego, ejecuta `pip install mysql-connector-python`.

Asegúrate de que la terminal muestra "base" y que en Jupyter, en la parte superior derecha, se muestra "base (y la versión de Python)". Después de estos pasos, vuelve a ejecutar la celda.

## Descripción de los Argumentos de *`connect()`:*

Antes de comenzar, exploremos algunos de los argumentos clave que utilizaremos al conectar a una base de datos utilizando el constructor `connect()`.

- `user`: Este argumento se refiere al nombre de usuario que se utilizará para autenticarse en el servidor MySQL.

- `password`: Aquí debemos proporcionar la contraseña correspondiente al usuario para la autenticación en el servidor MySQL.

- `database`: Es el nombre de la base de datos a la que deseamos conectarnos.

- `host`: Este argumento se utiliza para especificar el nombre del servidor MySQL o la dirección IP a la que nos conectaremos. Por defecto, este valor es "localhost", que se refiere a la misma máquina en la que se ejecuta el código (127.0.0.1 es la dirección IP equivalente).

- `port`: El puerto TCP/IP del servidor MySQL. Debe ser un número entero y, por defecto, se establece en 3306.


Existen más argumentos que son opcionales o que se usarán cuando queramos configurar la conexión de una manera muy concreta. Podéis consultar la descripción de todos los parámetros [aquí](https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html). 

In [ ]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np


In [ ]:
# ahora creamos la conexión con los argumentos:

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='tienda')

print(cnx)
cnx.close()

En ocasiones podemos sufrir errores de conexión, por lo que es interesante añadir manejo de excepciones al código anterior. Usando *errorcode* podemos crear casos para cada tipo de error posible:

In [ ]:
# En este código estamos haciendo un try except. Si recordamos esto nos permitía hacer un manejo de los errores, para evitar que nuestro código se pare. Para eso lo que estamos haciendo es
## intenta hacer la conexión son la base de datos de tienda 
try:
  cnx = mysql.connector.connect(user='root', password='admin',
                              host='127.0.0.1',
                              database='tienda')
# en caso de que no lo consigas por que hay algún error entonces ...
except mysql.connector.Error as err:

  # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  
  # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  
  # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
  else:
    print(err)
else:
  cnx.close()

## Desconexión de la base de datos usando MySQLConnector

Una vez hayamos terminado de realizar las consultas o trabajar de cualquier modo con la base de datos a la que nos hemos conectado con el connector, tendremos que desconectarnos usando el método *close()*:

In [ ]:
# En este caso vamos a pasarle los atributos al conector usando un diccionario:

config = {
  'user': 'root',
  'password': 'AlumnaAdalab',
  'host': '127.0.0.1',
  'database': 'tienda',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cnx.close()

En este fragmento de código, hemos establecido la conexión `cnx` con la base de datos "tienda". Con esta conexión, podemos llevar a cabo consultas sobre los datos almacenados en sus tablas (más adelante exploraremos cómo hacerlo). Cuando hayamos finalizado nuestras operaciones y deseemos desconectarnos, será necesario utilizar el método `close()` del objeto connector que hemos definido, en este caso llamado `cnx`. Es importante destacar que este método no requiere argumentos, lo que lo hace de uso sencillo.

# Realización de queries usando MySQLConnector:

Una vez que creamos la conexión con mysql, vamos a usar el método *cursor()* y *execute()* para poder realizar las queries.

El método cursor() se utiliza para crear un objeto de cursor, que es esencial para ejecutar consultas SQL en una conexión de base de datos. Una vez que tienes un objeto de cursor, puedes utilizar el método execute() para enviar una consulta SQL al servidor MySQL y ejecutarla.

In [ ]:
# realizamos la conexión con la BBDD que queremos
cnx = mysql.connector.connect(user='root', password='admin',
                              host='127.0.0.1',
                              database='tienda')

# iniciamos el cursor                               
mycursor = cnx.cursor()

# query de ejemplo:

query = ("""SELECT city, state FROM customers 
         WHERE customer_number BETWEEN 121 AND 124""")

# luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
## De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
mycursor.execute(query)

En el *execute()* podemos pasar cualquier tipo de query: crear tablas, alterar tablas, insertar datos,... vamos a verlo con unos ejemplos.

Vamos a aprovechar para introducir el manejo de excepciones a la hora de trabajar con MySQL Connector/Python.

- Crear Bases de Datos:

In [ ]:
cnx = mysql.connector.connect(user='root', password='admin',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE BD_pruebas")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

**Ojo:** al realizar la conexión, no le hemos indicado ninguna bases de datos, ya que la estamos creando.

En este ejemplo, estamos estableciendo la conexión y creando el *cursor*, después intentamos crear la base de datos "BD_pruebas". Si no se puede crear, nos dará un mensaje con el error que ha ocurrido.

Si ejecutamos dos veces esta celda, nos dará un aviso diendo que la base de datos ya existe.

- Creación de tablas:

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE TABLE customers (name VARCHAR(255), address VARCHAR(255))")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

Ahora sí hemos establecido la base de datos sobre la que estamos trabajando, por lo demás, este código es igual que el anterior, cambiando la query para crear una tabla en nuestra base de datos.

- Inserción de datos:

Ahora que hemos completado la creación de una base de datos, hemos definido una tabla con sus columnas y restricciones correspondientes, estamos listos para comenzar a agregar registros a esta tabla.

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
query = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = ("Ana", "Calle 21")
try: 
    mycursor.execute(query, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

Seguimos trabajando sobre BD_pruebas. Ahora hemos definido nuestra query para la inserción de datos en la que hemos incluido `%s` por cada uno de los valores que vamos a introducir. Después, definimos una variable en la que van los datos que queremos insertar en forma de tupla.

Esta vez, en el *execute()* hemos pasado dos argumentos: el primero con la query y el segundo con los valores.

**Uso de commit:** A la hora de trabajar con sentencias INSERT es necesario el uso de *commit()* para que los cambios se efectuen en la base de datos. De no llamar a ese método, las inserciones no se llevarán a cabo.

- Inserción de múltiples registros:

En el ejemplo anterior hemos introducido una única fila en nuestra base de datos. Si queremos insertar múltiples filas tenemos que usar *executemany()* y definir una variable con todos los datos que queremos insertar en forma de lista de tuplas.

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = [
  ('Ana', 'Lowstreet 4'),
  ('Rocio', 'Apple st 652'),
  ('Juana', 'Mountain 21'),
  ('Pedro', 'Valley 345')
]

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

- Eliminar registros de una tabla:

**Uso de rollback:** Si después de haber realizado una transacción con *execute()* nos diésemos cuenta de que los datos que hemos introducido son incorrectos en alguna manera, si aún no hemos ejecutado *commit()* podríamos dar marcha atrás y desestimar los cambios usando *rollback()*:

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = ("Lucia", "Plaza 22")
try:
    mycursor.execute(sql, val)
    cnx.rollback()
    print(mycursor.rowcount, "registro no insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

mycursor.execute("SELECT * FROM customers")  
results = mycursor.fetchall()
print(results)

Si ya hemos ejecutado *commit()* y queremos eliminar un registro de una tabla existente, usaremos *execute()* con la sentencia "DELETE FROM" de la siguiente manera:

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "DELETE FROM customers WHERE address = 'Calle 21'"
try:
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s eliminado/s")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

- Actualizar una tabla:

También podemos actualizar registros de la misma forma que lo haríamos en SQL con la sentencia "UPDATE":

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "UPDATE customers SET address = 'Canyon 123' WHERE address = 'Valley 345'"
try:
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s modificado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

**NOTA:** siempre que hagamos cualquier modificación sobre los registros tenemos que usar *commit()* para que esos cambios se hagan efectivos.

# Modos de acceso a los resultados de la consulta:

El uso de *execute()* convierte el *cursor* en un iterable al que podemos acceder de varias maneras. Podemos recorrer sus elementos uno por uno como si fuera una lista utilizando un bucle for. 

In [ ]:
# conectamos con el servidor
cnx = mysql.connector.connect(
              user='root', password='AlumnaAdalab',
              host='127.0.0.1',
              database='tienda'
)

# creamos el servidor
mycursor = cnx.cursor()

# ejecutamos la query para ver las tablas de la BBDD que hemos indicado en la conexión
mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

En el ejemplo vemos como iterando por el *cursor* hemos accedido a todos los resultados de la query uno por uno y nos lo devuelve en forma de tupla.
Sin embargo, MySQL Connector/Python también proporciona formas específicas de hacerlo de manera más eficiente y conveniente.

## Método fetchone()

Si deseamos acceder a la primera fila del resultado, podemos utilizar el método *fetchone()*. Este método recuperará la primera fila del resultado de la consulta y avanzará el cursor al siguiente registro. Como resultado, la próxima vez que utilicemos el método *fetchone()* en el mismo cursor, se mostrarán datos diferentes. Veámoslo con un ejemplo:

In [ ]:
# conectamos con el servidor
cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="admin",
  database="tienda"
)

# iniciamos el cursor
mycursor = cnx.cursor()

# ejecutamos nuestra query
mycursor.execute("SELECT * FROM employees")

# en este caso solo nos interesa el primer resultado de nuestra query, por lo que usaremos el método fetchone, para que python solo nos devuelva el primero y después printemos el resultado
myresult = mycursor.fetchone()
print("el resultado 1 es: ", myresult)

# volvemos a ejecutar el método fetchone, como ya nos mostró el primer resultado, y no hemos cerrado la conexión, al volver a ejecutarlo, nos mostrará el segundo resultado. Pero si nos fijamos, lo hace de uno en uno. 
myresult = mycursor.fetchone()
print("el resultado 2 es: ",myresult)

**Ojo:** si no cerramos la conexión, cada vez que ejecutemos *fetchone()* pasará al siguiente registro y nos irá devolviendo los datos uno a uno.

## Método fetchall()

En cambio, si deseamos seleccionar todos los resultados devueltos por una consulta SQL, podemos utilizar el método *fetchall()*:

In [ ]:
# conectamos con el servidor
cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="AlumnaAdalab",
  database="tienda"
)

# iniciamos el cursor
mycursor = cnx.cursor()

# realizamos nuestra query
mycursor.execute("SELECT * FROM employees")

# le decimos a Python que nos devuelva todos los resultados de la query usando el método fetchall
myresult = mycursor.fetchall()
print(myresult)

Podemos ver que nos devuelve una lista de tuplas, en la que cada tupla es una fila de nuestra consulta.

Si quisieramos acceder a cada registro de los resultados, podemos hacerlo usando myresult como un iterable. Esto nos devolverá cada fila resultado como una tupla:

In [ ]:
# conectamos con el servidor
cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="AlumnaAdalab",
  database="tienda"
)

# iniciamos el cursor
mycursor = cnx.cursor()

# realizamos la query
mycursor.execute("SELECT * FROM employees")

# le pedimos que almacene todos los resultados
myresult = mycursor.fetchall()

# accedemos a cada fila de una en una usando un bucle for. Fijaos que nos devuelve tuplas!!!
for x in myresult:
  print(x)

# Integración de resultados de MySQL Connector/Python con Pandas

**¿Qué es Pandas?**

Pandas es una destacada biblioteca de Python que se utiliza ampliamente en la gestión y manipulación de datos. Es especialmente útil cuando se trabaja con datos estructurados dispuestos en forma de tablas, como los que se almacenan en bases de datos SQL o hojas de cálculo de Excel. Pandas permite cargar, limpiar y procesar estos datos, extrayendo información valiosa y combinando diversas fuentes de datos, entre muchas otras tareas.
La estructura fundamental para trabajar con datos en Pandas se denomina DataFrame.

En esta sección de la lección, exploraremos cómo importar datos desde una base de datos MySQL a un DataFrame de Pandas y realizaremos algunas operaciones simples con estos datos. En lecciones futuras, profundizaremos en Pandas y su uso en detalle. El propósito de los ejemplos que siguen es brindarte una comprensión sólida de cómo integrar MySQL Connector/Python con Pandas.

## Crear un DataFrame de Pandas desde una sentencia SQL:

Como mencionamos anteriormente, Pandas ofrece una integración sencilla con una variedad de formatos de archivo y fuentes de datos, como archivos CSV, Excel, JSON y bases de datos SQL, entre otros. Por ejemplo, para cargar datos desde una base de datos SQL, puedes ejecutar el siguiente fragmento de código, que incluye una llamada al constructor de DataFrames:

In [ ]:
# primero tenemos que importar la libría

import pandas as pd

In [ ]:
# hacemos la conexión con el servidor
cnx = mysql.connector.connect(user='root', password='admin',
                              host='127.0.0.1',
                              database='tienda')

# iniciamos el cursor
mycursor = cnx.cursor()

# ejecutamos nuestra query
mycursor.execute("SELECT * FROM employees")

# le decimos que nos devuelva todos los resultados y los almacenamos en una variable llamada myresult
myresult = mycursor.fetchall()

#Creamos un dataframe con los resultados de la consulta SQL almacenados en myresult. Si os fijais le estamos pasando un parámetro llamado "columns" donde estamos especificando cuáles son las columnas de lo que será nuestro dataframe
df = pd.DataFrame(myresult, columns = ['ID', 'Nombre', 'Apellido','Email','Telefono','Direccion','Ciudad','Pais'])

#Cerramos la conexion
cnx.close()

In [ ]:
# mostramos las primeras filas del dataframe usando el método .head()

df.head()

Otra opción más directa para ver los resultados de nuestra query en un DataFrame consiste en encargarle a Pandas directamente la ejecución de la consulta SQL. Para ello usaremos el método *read_sql_query()* como vemos a continuación:

In [ ]:
# realizamos la conexión con el servidor
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='tienda')


# escribimos nuestra query
sql = "SELECT * FROM employees"

# utilizamos el método pd.read_sql_qury() para convertir los resultados de nuestra query en un DataFrame que podamos ver de forma amigable en Python
df = pd.read_sql_query(sql, cnx)

#Lo siguiente es un wrap up que funciona tanto con una tabla como con una consulta SQL.
#pd.read_sql(sql, cnx)

#Cerramos la conexion
cnx.close()

In [ ]:
# mostramos las primeras filas del dataframe usando el método .head()

df.head()

## Guardado de datos:

Una vez que tenemos nuestro Dataframe podemos guardarlo en diferentes formatos. En este caso lo vamos a guardar en *csv*:

In [ ]:
df.to_csv("fichero.csv")

Para poder guardarlo, usamos *.to_csv()* y especificamos el nombre de nuestro fichero con la extensión. Más adelante aprenderemos cómo guardar los datos en diferentes formatos.

## Intro Pandas

Pandas, la biblioteca de Python dedicada a la gestión y análisis de datos, se destaca por sus numerosas características fundamentales:

- Introduce innovadoras estructuras de datos basadas en los arrays de NumPy, enriqueciéndolas con capacidades avanzadas.

- Facilita la lectura y escritura de datos en formatos populares como CSV, Excel y bases de datos SQL, simplificando la manipulación de datos externos.

- Proporciona una accesibilidad excepcional a los datos a través de índices y etiquetas para filas y columnas, simplificando la navegación en conjuntos de datos complejos.

- Ofrece una amplia gama de métodos para reorganizar, dividir y fusionar conjuntos de datos, lo que agiliza la manipulación de datos según tus necesidades.

- Permite un manejo eficiente de series temporales, lo que resulta esencial para el análisis de datos relacionados con el tiempo.

En este apartado vamos a aprender a utilizar métodos como head(), tail(), describe(), entre otros, para obtener información relevante de los datos. Estos métodos te permitirán examinar rápidamente la estructura de tus datos y comprender sus estadísticas clave.


Empezamos por importar las librerías que vamos a usar:

### Principales métodos de Pandas:

- Métodos para exploración básica del *DataFrame*: 

    - `.head()`
    - `.tail()`
    - `.sample()`
    - `.describe()`
    - `.duplicated()`
    - `.isnull()`
    - `.isna()`
    - `.info()`
    - `.columns`

- `.head()`

Muestra las primeras líneas de nuestro Dataframe. Por defecto nos enseña 5.

In [ ]:
df.head()

In [ ]:
# si queremos ver un número diferente de filas, se lo podemos especificar

df.head(9)

- `.tail()`

Devuelve las últimas 5 filas del Dataframe. Al igual que con `.head()`, podemos especificar un número diferente.

In [ ]:
df.tail()

In [ ]:
df.tail(7)

- `.sample()`

Muestra filas aleatorias. Por defecto nos devuelve 1, pero también le podemos dar un valor diferente.

In [ ]:
df.sample()

In [ ]:
df.sample(4)

- `describe()`

Nos devuelve un *DataFrame* con un resumen de los principales estadísticos (media, mediana, desviacón estándar etc.) de nuestras **columnas numéricas**.

In [ ]:
df.describe()

Si queremos ver el `.describe()`de las variables categóricas, tenemos que incluir `include = "object"`. Podemos ver que nos devuelve diferentes estadísticas que para las numéricas.

- count: El número total de valores no nulos en la columna.
- unique: El número de valores únicos en la columna.
- top: El valor más frecuente en la columna.
- freq: La frecuencia del valor más común (el número de veces que aparece).

In [ ]:
df.describe(include="object")

- `duplicated()`

Devuelve una Serie booleana que indica si cada fila es un duplicado de una fila previamente vista.

In [ ]:
df.duplicated()

Si queremos ver el número de filas duplicadas de nuestro Dataframe tenemos que usar `.sum()`

In [ ]:
df.duplicated().sum()

- `isnull()` / - `isna()`:

Estos son los métodos para ver los valores nulos que tenemos en nuestros datos, podemos usar cualquiera de ellos ya que hacen lo mismo.

Nos devuelve un booleano con los valores que faltan en nuestro DataFrame.

In [ ]:
df.isnull()

Al igual que con los duplicados, podemos usar `.sum()` para poder entender mejor el resultado. Ahora nos devuelve el número de valores nulos por cada columna.

In [ ]:
df.isnull().sum()

- `info()`

El método `info()` proporciona un pequeño resumen que incluye:

- El total de columnas en el DataFrame.
- Los nombres de las columnas.
- La cantidad de valores no nulos en cada columna.
- El tipo de datos de cada columna en el DataFrame.

In [ ]:
df.info()

- `columns`

Nos muestra el nombre de las columnas de nuestro *DataFrame* .

In [ ]:
df.columns

## Ejercicios:

1. Haz las importaciones necesarias para establecer la conxión y trabaar con Pandas.Establece la conexión usando mysql.connector utilizando try/except por si la conexión devielve un error.

2. Crea un nuevo "schema" llamado "alumnas-promo-X"

3. Crea una tabla con la información de las alumnas de la promo. Debe incluir: id_alumna, nombre, apellidos, ciudad y teléfono.

4. Inserta por lo menos 10 datos en la tabla creada en el ejercicio anterior (pueden ser inventados).

5. Haz una consulta que devuelva todos los registros de la tabla "alumnas" y conviértelo en Dataframe (no te olvides del nombre de las columnas).

6. Muestra los 3 primeros y tres últimos registros del Dataframe.

7. ¿Hay registros duplicados?

### Bonus:

8. Haz una consulta para extraer las diferentes ciudades en las que hay alumnas.


9. Crea al menos dos Dataframes con la información de las alumnas de dos ciudades diferentes (usando dos queries).